In [1]:
import os
import numpy as np
import pandas as pd
from matchms import Spectrum
from matchms.similarity import ModifiedCosine

In [2]:
# import some functions, just run this cell

def generate_library_df(library_mgf):
    """
    Generate metadata dataframe for the mgf file
    """
    with open(f'input/{library_mgf}', 'r') as file:
        spectrum_list = []
        db_idx = 1
        for line in file:
            # empty line
            _line = line.strip()  # remove leading and trailing whitespace
            if not _line:
                continue
            elif line.startswith('BEGIN IONS'):
                spectrum = {}
                # initialize spectrum
                mz_list = []
                intensity_list = []
            elif line.startswith('END IONS'):
                if len(mz_list) == 0:
                    continue
                spectrum['mz_ls'] = mz_list
                spectrum['intensity_ls'] = intensity_list
                spectrum['db_idx'] = db_idx
                db_idx += 1
                spectrum_list.append(spectrum)
                continue
            else:
                # if line contains '=', it is a key-value pair
                if '=' in _line:
                    # split by first '=', in case of multiple '=' in the line
                    key, value = _line.split('=', 1)
                    spectrum[key] = value
                else:
                    # if no '=', it is a spectrum pair, split by tab or space
                    this_mz, this_int = _line.split()
                    try:
                        mz_list.append(float(this_mz))
                        intensity_list.append(float(this_int))
                    except:
                        continue

    df = pd.DataFrame(spectrum_list)

    # split adduct by '[' and ']', get the middle part
    df['_ADDUCT'] = df['ADDUCT'].apply(lambda x: x.split('[')[1].split(']')[0] if '[' in x else x)

    return df


def select_library(df, rt_tol=1, core_adduct_ls=None,
                   ms2_tol_da=0.05, modcos_score_cutoff=0.6, modcos_peak_cutoff=4):
    """
    Filter the library based on core adduct list, ion dependency, modcos match, and isobaric mass check
    """
    df['selected'] = [False] * df.shape[0]
    # get the unique molecules
    unique_smiles = df['SMILES'].unique().tolist()

    selected_scan_ls = []
    cmpd_adduct_to_be_removed_dict = {}  # key: RT, value: cmpd_adduct_to_be_removed

    df['RTINSECONDS'] = df['RTINSECONDS'].astype(float)
    df['binned_RT'] = pd.cut(df['RTINSECONDS'], bins=range(0, int(df['RTINSECONDS'].max()) + rt_tol * 2, rt_tol * 2))
    for smiles in unique_smiles:
        # filter the library for the molecule
        sub_df = df[(df['SMILES'] == smiles)]
        for binned_rt in sub_df['binned_RT'].unique():
            _subdf = sub_df[sub_df['binned_RT'] == binned_rt]
            # check the subdf
            scan_ls, cmpd_adduct_to_be_removed = subdf_check(_subdf, core_adduct_ls=core_adduct_ls,
                                                             ms2_tol_da=ms2_tol_da,
                                                             modcos_score_cutoff=modcos_score_cutoff,
                                                             modcos_peak_cutoff=modcos_peak_cutoff)
            # if len(scan_ls) < len(_subdf):
            #     print(f'{len(_subdf) - len(scan_ls)} spectra are removed for molecule {smiles}')
            selected_scan_ls.extend(scan_ls)
            if cmpd_adduct_to_be_removed:
                cmpd_adduct_to_be_removed_dict[binned_rt] = cmpd_adduct_to_be_removed

    # mark the selected spectra
    df.loc[df['db_idx'].isin(selected_scan_ls), 'selected'] = True

    # remove spectra, indicated by cmpd_adduct_to_be_removed_dict
    if cmpd_adduct_to_be_removed_dict:
        for binned_rt, cmpd_adduct_to_be_removed in cmpd_adduct_to_be_removed_dict.items():
            for cmpd_adduct in cmpd_adduct_to_be_removed:
                name, adduct, _ = cmpd_adduct.split(':')
                df.loc[(df['binned_RT'] == binned_rt) & (df['NAME'] == name.strip()) & (df['ADDUCT'] == adduct.strip()),
                'selected'] = False

    # remove cols
    df = df.drop(['_ADDUCT', 'db_idx', 'binned_RT'], axis=1)

    print(f'{df.shape[0]} spectra in the library')
    print(f'{df["selected"].sum()} spectra selected from the library')
    print(f'{len(df["NAME"].unique())} compounds in the library')

    return df


def write_to_mgf(df, out_mgf):
    """
    Write the selected library to a new mgf file
    """
    df = df[df['selected']].reset_index(drop=True)
    # remove cols
    df = df.drop(['selected'], axis=1)
    # move cols mz_ls and intensity_ls to the end
    df = df[[col for col in df.columns if col not in ['Num peaks', 'mz_ls', 'intensity_ls']] + ['Num peaks', 'mz_ls',
                                                                                                'intensity_ls']]

    with open(out_mgf, 'w') as file:
        for idx, row in df.iterrows():
            file.write('BEGIN IONS\n')
            for key, value in row.items():
                if key == 'mz_ls':
                    for mz, intensity in zip(row['mz_ls'], row['intensity_ls']):
                        file.write(f'{mz} {intensity}\n')
                elif key == 'intensity_ls':
                    continue
                else:
                    file.write(f'{key}={value}\n')
            file.write('END IONS\n\n')


def write_tsv(df, library_tsv, out_tsv):
    """
    Write the selected library to a new tsv file
    """
    lib_tsv = pd.read_csv(f'input/{library_tsv}', sep='\t')

    # all SCANs selected
    selected_scans = df[df['selected']]['SCANS'].tolist()

    # reserve lib_tsv with selected scans in 'EXTRACTSCAN' column
    lib_tsv = lib_tsv[lib_tsv['EXTRACTSCAN'].astype(str).isin(selected_scans)]
    lib_tsv.to_csv(out_tsv, sep='\t', index=False, na_rep='N/A')


def subdf_check(df, core_adduct_ls=None,
                ms2_tol_da=0.05, modcos_score_cutoff=0.6, modcos_peak_cutoff=4):
    """
    check the dataframe for one molecule
    :return: a list of selected scan numbers, a list of compound:adduct to be removed
    """
    if core_adduct_ls is None:
        core_adduct_ls = ['M+H', 'M-H2O+H', 'M+NH4', 'M-2H2O+H', 'M-H2O+NH4', 'M-2H2O+NH4', '2M+H',
                          '2M-H2O+H', '2M+NH4', '2M-2H2O+H', '2M-H2O+NH4', '2M-2H2O+NH4', 'M-H']

    # create a ModifiedCosine object
    modified_cosine = ModifiedCosine(tolerance=ms2_tol_da)

    # scan numbers of the selected spectra
    scan_no_ls = []

    subdf = df.copy()
    # check the core adducts, if any adduct in the core_adduct_ls is in the library
    subdf['core_adduct'] = subdf['_ADDUCT'].map(lambda x: x in core_adduct_ls)

    # if no core adducts are found, return empty list
    if not subdf['core_adduct'].any():
        # print('No core adducts found: ', subdf['NAME'].iloc[0], '   Found adducts: ',
        #       subdf['_ADDUCT'].unique().tolist())
        return scan_no_ls, None

    # select the spectra with core adducts
    core_df = subdf[subdf['core_adduct']].copy()
    # sort by core_adduct_ls
    core_df['core_adduct'] = core_df['_ADDUCT'].map(lambda x: core_adduct_ls.index(x))
    core_df = core_df.sort_values('core_adduct')
    # get all core spectra
    core_spec_ls = []
    for idx, row in core_df.iterrows():
        core_spec = Spectrum(mz=np.array(row['mz_ls']),
                             intensities=np.array(row['intensity_ls']),
                             metadata={'precursor_mz': row['PEPMASS'],
                                       'adduct': row['_ADDUCT']})
        core_spec_ls.append(core_spec)

    unique_adducts = subdf['_ADDUCT'].unique().tolist()
    # check the adduct dependency
    if len(unique_adducts) > 1:
        allowed_adducts = unique_adducts.copy()
        if 'M-H2O+Na' in unique_adducts and 'M-H2O+H' not in unique_adducts:
            allowed_adducts.remove('M-H2O+Na')
            if 'M-2H2O+Na' in allowed_adducts:
                allowed_adducts.remove('M-2H2O+Na')
            if 'M-3H2O+Na' in allowed_adducts:
                allowed_adducts.remove('M-3H2O+Na')
        if 'M-H2O+K' in unique_adducts and 'M-H2O+H' not in unique_adducts:
            allowed_adducts.remove('M-H2O+K')
            if 'M-2H2O+K' in allowed_adducts:
                allowed_adducts.remove('M-2H2O+K')
            if 'M-3H2O+K' in allowed_adducts:
                allowed_adducts.remove('M-3H2O+K')
        if 'M-2H2O+Na' in unique_adducts and 'M-H2O+Na' not in unique_adducts and 'M-2H2O+H' in unique_adducts:
            allowed_adducts.remove('M-2H2O+Na')
            if 'M-3H2O+Na' in allowed_adducts:
                allowed_adducts.remove('M-3H2O+Na')
        if 'M-2H2O+K' in unique_adducts and 'M-H2O+K' not in unique_adducts and 'M-2H2O+H' in unique_adducts:
            allowed_adducts.remove('M-2H2O+K')
            if 'M-3H2O+K' in allowed_adducts:
                allowed_adducts.remove('M-3H2O+K')
        if 'M-3H2O+Na' in unique_adducts and 'M-2H2O+Na' not in unique_adducts and 'M-3H2O+H' in unique_adducts:
            allowed_adducts.remove('M-3H2O+Na')
        if 'M-3H2O+K' in unique_adducts and 'M-2H2O+K' not in unique_adducts and 'M-3H2O+H' in unique_adducts:
            allowed_adducts.remove('M-3H2O+K')
        if 'M-2H2O+H' in unique_adducts and 'M-H2O+H' not in unique_adducts and 'M+H' in unique_adducts:
            allowed_adducts.remove('M-2H2O+H')
            if 'M-3H2O+H' in allowed_adducts:
                allowed_adducts.remove('M-3H2O+H')
        if 'M-3H2O+H' in unique_adducts and 'M-2H2O+H' not in unique_adducts and 'M+H' in unique_adducts:
            allowed_adducts.remove('M-3H2O+H')
        if 'M-2H2O+NH4' in unique_adducts and 'M-H2O+NH4' not in unique_adducts and 'M-2H2O+H' not in unique_adducts:
            allowed_adducts.remove('M-2H2O+NH4')
            if 'M-3H2O+NH4' in allowed_adducts:
                allowed_adducts.remove('M-3H2O+NH4')
        if 'M-3H2O+NH4' in unique_adducts and 'M-2H2O+NH4' not in unique_adducts and 'M-3H2O+H' not in unique_adducts:
            allowed_adducts.remove('M-3H2O+NH4')

        if '2M-H2O+Na' in unique_adducts and '2M-H2O+H' not in unique_adducts:
            allowed_adducts.remove('2M-H2O+Na')
            if '2M-2H2O+Na' in allowed_adducts:
                allowed_adducts.remove('2M-2H2O+Na')
            if '2M-3H2O+Na' in allowed_adducts:
                allowed_adducts.remove('2M-3H2O+Na')
        if '2M-H2O+K' in unique_adducts and '2M-H2O+H' not in unique_adducts:
            allowed_adducts.remove('2M-H2O+K')
            if '2M-2H2O+K' in allowed_adducts:
                allowed_adducts.remove('2M-2H2O+K')
            if '2M-3H2O+K' in allowed_adducts:
                allowed_adducts.remove('2M-3H2O+K')
        if '2M-2H2O+Na' in unique_adducts and '2M-H2O+Na' not in unique_adducts and '2M-2H2O+H' in unique_adducts:
            allowed_adducts.remove('2M-2H2O+Na')
            if '2M-3H2O+Na' in allowed_adducts:
                allowed_adducts.remove('2M-3H2O+Na')
        if '2M-2H2O+K' in unique_adducts and '2M-H2O+K' not in unique_adducts and '2M-2H2O+H' in unique_adducts:
            allowed_adducts.remove('2M-2H2O+K')
            if '2M-3H2O+K' in allowed_adducts:
                allowed_adducts.remove('2M-3H2O+K')
        if '2M-3H2O+Na' in unique_adducts and '2M-2H2O+Na' not in unique_adducts and '2M-3H2O+H' in unique_adducts:
            allowed_adducts.remove('2M-3H2O+Na')
        if '2M-3H2O+K' in unique_adducts and '2M-2H2O+K' not in unique_adducts and '2M-3H2O+H' in unique_adducts:
            allowed_adducts.remove('2M-3H2O+K')
        if '2M-2H2O+H' in unique_adducts and '2M-H2O+H' not in unique_adducts and '2M+H' in unique_adducts:
            allowed_adducts.remove('2M-2H2O+H')
            if '2M-3H2O+H' in allowed_adducts:
                allowed_adducts.remove('2M-3H2O+H')
        if '2M-3H2O+H' in unique_adducts and '2M-2H2O+H' not in unique_adducts and '2M+H' in unique_adducts:
            allowed_adducts.remove('2M-3H2O+H')
        if '2M-2H2O+NH4' in unique_adducts and '2M-H2O+NH4' not in unique_adducts and '2M-2H2O+H' not in unique_adducts:
            allowed_adducts.remove('2M-2H2O+NH4')
            if '2M-3H2O+NH4' in allowed_adducts:
                allowed_adducts.remove('2M-3H2O+NH4')
        if '2M-3H2O+NH4' in unique_adducts and '2M-2H2O+NH4' not in unique_adducts and '2M-3H2O+H' not in unique_adducts:
            allowed_adducts.remove('2M-3H2O+NH4')

        subdf['selected'] = subdf['_ADDUCT'].map(lambda x: x in allowed_adducts)

        # for unselected spectra, calculate the scores for different adducts
        for idx, row in subdf.iterrows():
            if not row['selected']:
                this_spec = Spectrum(mz=np.array(row['mz_ls']),
                                     intensities=np.array(row['intensity_ls']),
                                     metadata={'precursor_mz': row['PEPMASS'],
                                               'adduct': row['_ADDUCT']})
                # calculate the ModifiedCosine score
                for core_spec in core_spec_ls:
                    if core_spec.get('adduct') == this_spec.get('adduct'):
                        continue
                    score = modified_cosine.pair(core_spec, this_spec)
                    if score['score'] >= modcos_score_cutoff or score['matches'] >= modcos_peak_cutoff:
                        subdf.loc[idx, 'selected'] = True
                        break

        # use spectrum with unambiguous compound to filter out the ambiguous ones
        cmpd_adduct_to_be_removed = None
        _subdf = subdf[(subdf['selected']) & (subdf['OTHER_MATCHED_COMPOUNDS'].isna())]
        if len(_subdf) > 0:
            cmpd_adduct_to_be_removed = subdf['OTHER_MATCHED_COMPOUNDS_NAMES'].tolist()
            # remove nan
            cmpd_adduct_to_be_removed = [x for x in cmpd_adduct_to_be_removed if str(x) != 'nan']
            # remove empty string
            cmpd_adduct_to_be_removed = [x for x in cmpd_adduct_to_be_removed if x]
            # split by ';' and flatten the list
            cmpd_adduct_to_be_removed = [x.split(';') for x in cmpd_adduct_to_be_removed]
            cmpd_adduct_to_be_removed = [item for sublist in cmpd_adduct_to_be_removed for item in sublist]

        # db_idx of the selected spectra
        scan_no_ls = subdf[subdf['selected']]['db_idx'].tolist()
        return scan_no_ls, cmpd_adduct_to_be_removed
    else:
        return subdf['db_idx'].tolist(), None


In [3]:
# create the output folder
if not os.path.exists('output'):
    os.makedirs('output')

# list all the files in the input folder
all_files = os.listdir('input')
# filter the mgf files
library_mgfs = [x for x in all_files if x.endswith('.mgf')]

for library_mgf in library_mgfs:
    print(f'Processing {library_mgf}')
    # check the existence of the library_mgf.tsv
    library_tsv = library_mgf.split('.mgf')[0] + '.tsv'
    if not os.path.exists(f'input/{library_tsv}'):
        print(f'{library_tsv} does not exist')
        continue
    df = generate_library_df(library_mgf)
    df = select_library(df, rt_tol=2, ms2_tol_da=0.05,
                        modcos_score_cutoff=0.6, modcos_peak_cutoff=4)

    out_mgf = 'output/' + library_mgf.split('.mgf')[0] + '_filtered.mgf'
    write_to_mgf(df, out_mgf)
    out_tsv = 'output/' + library_tsv.split('.tsv')[0] + '_filtered.tsv'
    write_tsv(df, library_tsv, out_tsv)

Processing 20240430_IM_BA_new_core_MZMine_libraryoutput_for_GNPS.mgf
6984 spectra in the library
6510 spectra selected from the library
1637 compounds in the library
